### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [3]:
import sys
import os
sys.path.append(os.path.abspath(".."))  # or "." if script is in root

from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [8]:
from source_files.azure_llm import get_client

model_client = get_client()

### Second concept: The Message

In [27]:
from autogen_agentchat.messages import TextMessage

message = TextMessage(content="I'd like to go to beirut", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 21, 51, 32, 675447, tzinfo=datetime.timezone.utc), content="I'd like to go to beirut", type='TextMessage')

### Third concept: The Agent

In [28]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

### Put it all together with on_messages

In [29]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

"Perfect! Pack your sunscreen and a good appetite, because you're about to love life (and the food) in Beirut. Want me to book a flight, or are you just dreaming of hummus heaven right now? 😎✈️"

### Let's make a local database of ticket prices

In [30]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

In [31]:
# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [37]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)
save_city_price("Beirut", 1000)

In [38]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [39]:
get_city_price("Rome")

499.0

In [46]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the the price of a two roundtrip tickets. The prices are for one roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [47]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_veDFSF7xmGVg4giypS7NFsjz', arguments='{"city_name":"Beirut"}', name='get_city_price')]
[FunctionExecutionResult(content='1000.0', name='get_city_price', call_id='call_veDFSF7xmGVg4giypS7NFsjz', is_error=False)]


"Beirut is calling, and it's only $1000 for a roundtrip ticket! Grab your passport and let's go before someone else steals your seat! So, for two roundtrip tickets, you'd be looking at $2000. 🌍✈️"